In [ ]:
#can use verilog internal random functions (and thus not fixing IN_WIDTH)

In [ ]:
#uses MatrixAdd_#M_#N_#pvPV_SI_HRx#hrf_(NIR, IR)_(NIL, IL) - v1.2

In [ ]:
import sys
import math
import random

In [ ]:
def MatrixAdd_SI_testbench_F(parameters, Print_To_File, Files_Location, IN_WIDTH):

    M = parameters["M"]
    N = parameters["N"]
    PR = parameters["PR"]
    HRR = parameters["HRR"]
    
    Enable_Input_Registers = parameters["ENABLE_INPUT_REGISTERS"]
    
    Enable_Input_Latch = parameters["ENABLE_INPUT_LATCH"]

    Enable_Extra_Data = 1
    
    NAdd = math.ceil(N/HRR);
    while math.ceil(N/NAdd) != HRR:
        HRR -= 1
    
    RS=math.ceil(M/PR)

    ModuleName="MatrixAdd_{}_{}_{}PV_SI_HRx{}_".format(M, N, PR, HRR)
    if Enable_Input_Registers<=0:
        ModuleName+="N"
    ModuleName+="IR_"
    if Enable_Input_Latch<=0:
        ModuleName+="N"
    ModuleName+="IL"
    UUTModuleName = ModuleName
    ModuleName += "_test_IW{}".format(IN_WIDTH)

    if Print_To_File<=0:
        of = sys.stdout
    else:
        of = open(Files_Location+UUTModuleName+"/"+ModuleName+".v", 'w+')

    print("`timescale 1ns / 500ps\n", file=of)
    print("module "+ModuleName+";", file=of)
    print("//IN_WIDTH fixed to {}".format(IN_WIDTH), file=of)
    print(file=of)

    print("reg clk, reset, enable;", file=of)
    print("wire readyForNewVectorStart;", file=of)
    lghrr=math.ceil(math.log2(HRR))
    if lghrr==1:
        print("wire vectorInSeries;", file=of)
    elif lghrr>1:
        print("wire [{}:0] vectorInSeries;".format(lghrr-1), file=of)
    RS=math.ceil(M/PR)
    lgRS=math.ceil(math.log2(RS))
    if lgRS<=1:
        print("wire vectorSetInNo;", file=of)
    else:
        print("wire [{}:0] vectorSetInNo;".format(lgRS-1), file=of)
    print("reg inReady;", file=of)
    print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
    for j in range(PR):
        for i in range(N):
            print("A{}V{}".format(i, j), end='', file=of)
            if j==(PR-1) and i==(N-1):
                print(";", file=of)
            elif i!=(N-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    #     print(file=of)
    print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
    for j in range(PR):
        for i in range(N):
            print("B{}V{}".format(i, j), end='', file=of)
            if j==(PR-1) and i==(N-1):
                print(";", file=of)
            elif i!=(N-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    #     print(file=of)
    ar=RS*PR-M
    ME = PR-ar-1
    if ME==0:
        st1="V0"
    else:
        st1="V0toV{}".format(ME)
    print("wire ", end='', file=of)
    NE = N - NAdd*(HRR-1)
    while NE<=0:
        NE += NAdd
    if NE==1:
        print(st1, end='', file=of)
        print("S0outReady;", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}outReady;".format(NE-1), file=of)
    print("wire ", end='', file=of)
    if NE==NAdd:
        print("VSNoutReady1; //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}outReady;".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}outReady;".format(NE, NAdd-1), file=of)
    if ar==0:
        print("wire VNoutReady1; //not used", file=of)
        print("wire VNoutReady2; //not used", file=of)
    else:
        if ME==(PR-2):
            st2="V{}".format(PR-1)
        else:
            st2="V{}toV{}".format(ME+1, PR-1)
        print("wire ", end='', file=of)
        if NE==1:
            print(st2, end='', file=of)
            print("S0outReady;", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}outReady;".format(NE-1), file=of)
        print("wire ", end='', file=of)
        if NE==NAdd:
            print("VSNoutReady2; //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}outReady;".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}outReady;".format(NE, NAdd-1), file=of)
    if lghrr==1:
        print("wire vectorOutSeries;", file=of)
    elif lghrr>1:
        print("wire [{}:0] vectorOutSeries;".format(lghrr-1), file=of)
    if lgRS<=1:
        print("wire vectorSetOutNo;", file=of)
    else:
        print("wire [{}:0] vectorSetOutNo;".format(lgRS-1), file=of)
    print("wire signed [{}:0] ".format(IN_WIDTH), end='', file=of)
    for j in range(PR):
        for i in range(NAdd):
            print("S{}V{}".format(i, j), end='', file=of)
            if j==(PR-1) and i==(NAdd-1):
                print(";", file=of)
            elif i!=(NAdd-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    #     print(file=of)
    print("wire ", end='', file=of)
    if NE==1:
        print(st1, end='', file=of)
        print("S0earlyOutReady;", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}earlyOutReady;".format(NE-1), file=of)
    print("wire ", end='', file=of)
    if NE==NAdd:
        print("CSNearlyOutReady1; //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}earlyOutReady;".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}earlyOutReady;".format(NE, NAdd-1), file=of)
    if ar==0:
        print("wire CNearlyOutReady1; //not used", file=of)
        print("wire CNearlyOutReady2; //not used", file=of)
    else:
        print("wire ", end='', file=of)
        if NE==1:
            print(st2, end='', file=of)
            print("S0earlyOutReady;", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}earlyOutReady;".format(NE-1), file=of)
        print("wire ", end='', file=of)
        if NE==NAdd:
            print("CSNearlyOutReady2; //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}earlyOutReady;".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}earlyOutReady;".format(NE, NAdd-1), file=of)

    print(UUTModuleName, file=of)
    print("#({})".format(IN_WIDTH), file=of)
    print("UUT(", file=of)

    print("clk, reset, enable,", file=of)
    print("readyForNewVectorStart,", file=of)
    print("vectorInSeries,", file=of)
    print("vectorSetInNo,", file=of)
    print("inReady,", file=of)
    for j in range(PR):
        for i in range(N):
            print("A{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    for j in range(PR):
        for i in range(N):
            print("B{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    if NE==1:
        print(st1, end='', file=of)
        print("S0outReady,", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}outReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("VSNoutReady1, //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}outReady,".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    if ar==0:
        print("VNoutReady1, //not used", file=of)
        print("VNoutReady2, //not used", file=of)
    else:
        if NE==1:
            print(st2, end='', file=of)
            print("S0outReady,", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}outReady,".format(NE-1), file=of)
        if NE==NAdd:
            print("VSNoutReady2, //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}outReady,".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    print("vectorOutSeries,", file=of)
    print("vectorSetOutNo,", file=of)
    for j in range(PR):
        for i in range(NAdd):
            print("S{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    if NE==1:
        print(st1, end='', file=of)
        print("S0earlyOutReady,", file=of)
    else:
        print(st1, end='', file=of)
        print("S0toS{}earlyOutReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("CSNearlyOutReady1, //not used", file=of)
    elif NE==NAdd-1:
        print(st1, end='', file=of)
        print("S{}earlyOutReady,".format(NE), file=of)
    else:
        print(st1, end='', file=of)
        print("S{}toS{}earlyOutReady,".format(NE, NAdd-1), file=of)
    if ar==0:
        print("CNearlyOutReady1, //not used", file=of)
        print("CNearlyOutReady2 //not used", file=of)
    else:
        if NE==1:
            print(st2, end='', file=of)
            print("S0earlyOutReady,", file=of)
        else:
            print(st2, end='', file=of)
            print("S0toS{}earlyOutReady,".format(NE-1), file=of)
        if NE==NAdd:
            print("CSNearlyOutReady2 //not used", file=of)
        elif NE==NAdd-1:
            print(st2, end='', file=of)
            print("S{}earlyOutReady".format(NE), file=of)
        else:
            print(st2, end='', file=of)
            print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
    print(");\n", file=of)

    InputRangeStart = -2**(IN_WIDTH-1)
    InputRangeEnd = 2**(IN_WIDTH-1)
    matrix_A = [[random.randrange(InputRangeStart, InputRangeEnd) for nn in range(N)] for jj in range(M)]
    matrix_B = [[random.randrange(InputRangeStart, InputRangeEnd) for nn in range(N)] for jj in range(M)]

    # print(matrix_A)
    # print(matrix_B)

    matrix_Res = [[0 for nn in range(N)] for jj in range(M)]
    for jj in range(M):
        for nn in range(N):
            matrix_Res[jj][nn] = matrix_A[jj][nn]+matrix_B[jj][nn]
    # print(matrix_Res)

    print("initial begin", file=of)
    print("\tclk = 1;", file=of)
    print("\treset <= 0;", file=of)
    print("\tenable <= 1;", file=of)
    print("\tinReady <= 1;", file=of)

    FirstInput = True
    for j in range(RS):
        if FirstInput:
            FirstInput = False
        elif Enable_Extra_Data and Enable_Input_Latch:
            print("\t#1;", file=of)
        else:
            print("\t#{};".format(HRR), file=of)

        #print(file=of)
        print("\t//R", end='', file=of)
        for jj in range(j*PR, (j+1)*PR):
            if jj<M:
                print(" {}".format(jj), end='', file=of)
        print(file=of)

        jc=0
        for jj in range(j*PR, (j+1)*PR):
            if jj<M:
                for nn in range(N):
                    print("\tA{}V{} <= {};".format(nn, jc, matrix_A[jj][nn]), file=of)
                    cA = matrix_A[jj][nn]
                    if cA >= 0:
                        print("\tA{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cA), file=of)
                    else:
                        print("\tA{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cA)), file=of)                        
                jc += 1
            else:
                break

        jc=0
        for jj in range(j*PR, (j+1)*PR):
            if jj<M:
                for nn in range(N):
                    cB = matrix_B[jj][nn]
                    if cB >= 0:
                        print("\tB{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cB), file=of)
                    else:
                        print("\tB{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cB)), file=of)                        
                jc += 1
            else:
                break

        if Enable_Extra_Data and Enable_Input_Latch:
            for h in range(HRR-1):
                print("\t#1;", file=of)
                print("\t//Extra Data:", file=of)
                jc=0
                for jj in range(j*PR, (j+1)*PR):
                    if jj<M:
                        for nn in range(N):
                            cR = random.randrange(InputRangeStart, InputRangeEnd)
                            if cR >= 0:
                                print("\tA{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                            else:
                                print("\tA{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)
                        jc += 1
                    else:
                        break
                jc=0
                for jj in range(j*PR, (j+1)*PR):
                    if jj<M:
                        for nn in range(N):
                            cR = random.randrange(InputRangeStart, InputRangeEnd)
                            if cR >= 0:
                                print("\tB{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                            else:
                                print("\tB{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)
                        jc += 1
                    else:
                        break

    print("end", file=of)
    print(file=of)

    print("always #0.5 clk = ~ clk;", file=of)
    print(file=of)

    initial_delay_p = 0
    print("//initial delay:", file=of)
    if Enable_Input_Registers<=0:
        print("//NIR: ", end='', file=of)
        print("0", file=of)
    else:
        print("//IR: ", end='', file=of)
        print("1", file=of)
        initial_delay_p += 1
    print("//Registered2to1Adder: 1", file=of)
    initial_delay_p += 1
    print("parameter initial_delay = {};".format(initial_delay_p), file=of)
    print(file=of)

    print("integer outfile;", file=of)
    print(file=of)

    print("task Print_Parameters_Close;", file=of)
    print("\tbegin", file=of)
    print("\t$fdisplay(outfile, \"\\nParameters' values:\");", file=of)
    print("\t$fdisplay(outfile, \"IN_WIDTH : fixed to {}\");".format(IN_WIDTH), file=of)
    print("\t$fclose(outfile);", file=of)
    print("\tend", file=of)
    print("endtask", file=of)
    print(file=of)

    print("task Simulation_Fail;", file=of)
    print("\tbegin", file=of)
    print("\t$fdisplay(outfile, \"Failed\");", file=of)
    print("\t$fdisplay(outfile, \"Error at %t\", $realtime);", file=of)
    print("\tPrint_Parameters_Close();", file=of)
    print("\t$display(\"Failed\");", file=of)
    print("\tend", file=of)
    print("endtask", file=of)
    print(file=of)

    print("initial begin", file=of)
    print("$timeformat (-9, 1, \" ns\", 0);", file=of)
    print("outfile = $fopen(\"_{}_Simulation_Result.txt\", \"w\");".format(IN_WIDTH), file=of)
    print("#initial_delay;", file=of)
    print("#0.5;", file=of)

    FirstOutputSeries = True
    for j in range(RS):
        for h in range(HRR):
            if FirstOutputSeries:
                FirstOutputSeries = False
            else:
                print("#1;", file=of)
            jc=0
            for jj in range(j*PR, (j+1)*PR):
                if jj<M:
                    print("if(", end='', file=of)
                    nc=0
                    FirstOutput = True
                    for nn in range(h*NAdd, (h+1)*NAdd):
                        if nn<N:
                            if FirstOutput:
                                FirstOutput = False
                            else:
                                print(" || ", end='', file=of)
                            cRes = matrix_Res[jj][nn]
                            if cRes >= 0:
                                print("S{}V{}!={}'d{}".format(nc, jc, IN_WIDTH+1, cRes), end='', file=of)
                            else:
                                print("S{}V{}!=-{}'d{}".format(nc, jc, IN_WIDTH+1, abs(cRes)), end='', file=of)
                            nc += 1
                        else:
                            break
                    print(") begin", file=of)
                    print("\tSimulation_Fail();", file=of)
                    print("\t$finish;", file=of)
                    print("end", file=of)
                    jc += 1
                else:
                    break

    print("#0.5;", file=of)
    print("$fdisplay(outfile, \"Succeeded\");", file=of)
    print("$fdisplay(outfile, \"Simulation completed without error at %t\", $realtime);", file=of)
    print("Print_Parameters_Close();", file=of)
    print("$display(\"Succeeded\");", file=of)
    print("$finish;", file=of)
    print("end", file=of)
    print(file=of)
    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()